#### This notebook deals with the way to do away with the inclusion of land points in the horizontal interpolation, so we are looking to mask the data before we do the interpolation (also we would be doing it in 2 steps instead of one to achieve a slightly higher accuracy and smoothness of data points near the coast so we donot have a noisy signal in the NEMO output near the coastlines)

#### We choose to tackle the problem in 4 steps

1) Mask out JP's land using JP's t mask (and u, v masks for his u and v data not in IC but in BC later)

2) Do a gridata linear interpolation to all our grid points (WCVI). After this step we get NAN over a handful of WCVI points.

3) Do a gridata nearest neighbour interpolation only to the places where we have NAN values right now in our WCVI domain.

4) Multiply the whole thing with our tmask (and u, v masks for the u,v data) from one of our mesh_mask.nc

In [1]:
import numpy as np
import netCDF4 as nc
import xarray as xr
from scipy.interpolate import griddata, interp1d
from salishsea_tools import nc_tools,viz_tools
import matplotlib.pyplot as plt
import matplotlib.cm as cm

from fancyimpute import KNN

In [2]:
#### Load JP's data 

fname = '/home/ssahu/saurav/JP_BC/cat_42_days_T.nc';

gridT = xr.open_dataset(fname);

In [3]:
gridT.keys

<bound method Mapping.keys of <xarray.Dataset>
Dimensions:            (bnds: 2, deptht: 50, time_counter: 43, x: 715, y: 1021)
Coordinates:
  * deptht             (deptht) float32 0.494025 1.54138 2.64567 3.81949 ...
  * time_counter       (time_counter) datetime64[ns] 2016-04-01T12:00:00 ...
  * y                  (y) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * x                  (x) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 ...
  * bnds               (bnds) int64 0 1
Data variables:
    nav_lon            (y, x) float32 -142.283 -142.256 -142.228 -142.2 ...
    nav_lat            (y, x) float32 45.0491 45.0479 45.0467 45.0456 ...
    time_counter_bnds  (time_counter, bnds) float64 4.873e+07 4.881e+07 ...
    votemper           (time_counter, deptht, y, x) float64 0.0 8.881 8.905 ...
    vosaline           (time_counter, deptht, y, x) float64 0.0 33.14 33.13 ...
    sosstsst           (time_counter, y, x) float64 0.0 8.881 8.905 8.925 ...
    sosaline           (

In [4]:
#### Load JP's mask

mask = nc.Dataset('/data/mdunphy/NEP036-N30-OUT/INV/mesh_mask.nc');

tmask_JP = mask.variables['tmask'][0,:,55:400,446:701];
umask_JP = mask.variables['umask'][0,:,55:400,446:701];
vmask_JP = mask.variables['vmask'][0,:,55:400,446:701];

print(tmask_JP.shape, umask_JP.shape, vmask_JP.shape)

(50, 345, 255) (50, 345, 255) (50, 345, 255)


### STEP 1) Mask JP's data

In [5]:
#### Slice out our domain of WCVI from JP and mask it at the same time saying the zero values as NAN

votemper_JP = np.array(gridT['votemper'][0,:,55:400,446:701]);
votemper_JP[...] = np.ma.masked_array(votemper_JP[...], mask = tmask_JP[...]);
votemper_JP[votemper_JP == 0] =['Nan'];

vosaline_JP = np.array(gridT['vosaline'][0,:,55:400,446:701]);
vosaline_JP[...] = np.ma.masked_array(vosaline_JP[...], mask = tmask_JP[...]);
vosaline_JP[vosaline_JP == 0] = ['Nan'];

glamt_bc_JP = np.array(gridT['nav_lon'][55:400,446:701]);
gphit_bc_JP = np.array(gridT['nav_lat'][55:400,446:701]);
deptht_JP = np.array(gridT['deptht'][:]);


In [7]:
vosaline_JP[:,0,0]

array([ 32.85047913,  32.8494606 ,  32.84786987,  32.84399796,
        32.84130096,  32.84194183,  32.8444519 ,  32.84753799,
        32.85254669,  32.85647964,  32.8595314 ,  32.86235046,
        32.86702728,  32.87555313,  32.88618469,  32.89451218,
        32.90081024,  32.90483093,  32.90930176,  32.92013931,
        32.95475769,  33.06747437,  33.2415123 ,  33.47072601,
        33.66275787,  33.83255005,  33.93229294,  33.96300888,
        33.97008896,  33.98950577,  34.0399704 ,  34.10607147,
        34.17634964,  34.25790787,  34.33724213,  34.39661026,
        34.45060349,  34.49922562,  34.54627991,  34.58870316,
        34.61618042,          nan,          nan,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan])

In [8]:
vosaline_JP[np.where(np.isnan(vosaline_JP))]=0;

In [9]:
vosaline_JP[:,0,0]

array([ 32.85047913,  32.8494606 ,  32.84786987,  32.84399796,
        32.84130096,  32.84194183,  32.8444519 ,  32.84753799,
        32.85254669,  32.85647964,  32.8595314 ,  32.86235046,
        32.86702728,  32.87555313,  32.88618469,  32.89451218,
        32.90081024,  32.90483093,  32.90930176,  32.92013931,
        32.95475769,  33.06747437,  33.2415123 ,  33.47072601,
        33.66275787,  33.83255005,  33.93229294,  33.96300888,
        33.97008896,  33.98950577,  34.0399704 ,  34.10607147,
        34.17634964,  34.25790787,  34.33724213,  34.39661026,
        34.45060349,  34.49922562,  34.54627991,  34.58870316,
        34.61618042,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ,   0.        ,   0.        ,
         0.        ,   0.        ])

#### STEP 2) First of the 2 step interpolation (linearly interpolate to our WCVI grid points) 

In [7]:
#### Load the WCVI points

fname_wcvi = '/ocean/ssahu/CANYONS/wcvi/grid/coordinates.nc'

with nc.Dataset(fname_wcvi, 'r') as coord:
    gphit_wcvi = coord.variables['gphit'][0,...];
    glamt_wcvi =  coord.variables['glamt'][0,...]; 
    
print((glamt_bc_JP[0,0],gphit_bc_JP[0,0]), (glamt_wcvi[0,0],gphit_wcvi[0,0]))

(-129.93523, 45.455914) (-127.00188120221485, 46.113550613066906)


In [8]:
X = glamt_bc_JP.flatten();

Y = gphit_bc_JP.flatten();

points = (X[:],Y[:]);

xi = (glamt_wcvi.flatten(), gphit_wcvi.flatten());


votemper_ic = np.zeros((votemper_JP.shape[0], glamt_wcvi.shape[0], glamt_wcvi.shape[1]));
vosaline_ic = np.zeros((vosaline_JP.shape[0], glamt_wcvi.shape[0], glamt_wcvi.shape[1]));

for i,j in enumerate(votemper_JP[:,...]):
    votemper_ic[i,...] = np.reshape(griddata(points, votemper_JP[i,...].flatten(), xi, method= 'linear'), glamt_wcvi.shape)
    vosaline_ic[i,...] = np.reshape(griddata(points, vosaline_JP[i,...].flatten(), xi, method= 'linear'), glamt_wcvi.shape)



In [9]:
vosaline_ic[:,0,0]

array([ 32.47419656,  32.47339993,  32.472351  ,  32.47062415,
        32.46957545,  32.46912266,  32.46870302,  32.46790199,
        32.46697466,  32.46655073,  32.46634141,  32.4661859 ,
        32.4660519 ,  32.4658899 ,  32.46588335,  32.46623902,
        32.46850987,  32.47960524,  32.52179053,  32.63602269,
        32.85181797,  33.12211982,  33.38578935,  33.65309569,
        33.81709729,  33.93756474,  34.00150822,  34.03875474,
        34.0547015 ,  34.05655214,  34.07886744,  34.13459396,
        34.20362748,  34.26293978,  34.33264525,  34.39530034,
        34.4533169 ,  34.49530198,  34.5369522 ,  34.579813  ,
        34.62248934,  34.63830741,  34.64125478,          nan,
                nan,          nan,          nan,          nan,
                nan,          nan])

#### STEP 3) Fill up the Nan values in the votemper and vosaline arrays of WCVI using the nearest neighbours

In [10]:
for i,j in enumerate(votemper_ic[:,...]):
    votemper_ic[i,...] = KNN(k=3).complete(votemper_ic[i,...]);
    vosaline_ic[i,...] = KNN(k=3).complete(vosaline_ic[i,...]);

Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 4 missing, elapsed time: 0.005
Imputing row 1/100 with 4 missing, elapsed time: 0.005
Imputing row 1/100 with 4 missing, elapsed time: 0.005
Imputing row 1/100 with 4 missing, elapsed time: 0.005
Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 4 missing, elapsed time: 0.006
Imputing row 1/100 with 8 missing, elapsed time: 0.005
Imputing row 1/100 with 8 missing, elapsed time: 0.006
Imputing row 1/100 with 9 missing, elapsed time: 0.006
Imputing row 1/100 with 9 missing, elapsed time: 0.006
Imputing row 1/100 with 9 missing, elapsed time: 0.006
Imputing row 1/100 with 9 missing, elapsed time: 0.005
Imputing r

ValueError: Input matrix must have some non-missing values

#### STEP 4) Multiply with our tmask from our mesh_mask to get the final data (This we will do after doing the vertical interpolation onto our depth levels and for now we will create the temporary nc file that we were making)

In [ ]:
i=4;

votemper_ic[np.where(np.isnan(votemper_ic[i,...]))] = np.reshape(griddata(points, votemper_ic[i,np.where(votemper_ic[~isnan(votemper_ic[i,...]))])].flatten(), xi, method= 'nearest')), \
                                                                   glamt_wcvi.shape)

In [ ]:
votemper_ic[4,:,:]

In [ ]:
votemper_JP.shape

In [ ]:
glamt_wcvi.shape

In [ ]:
votemper_ic.shape

In [ ]:
len(Y)

In [ ]:
np.where(np.isnan(votemper_ic_imuted[4,...]))

In [ ]:
np.mean(votemper_ic[0,2:7,4:8])

In [ ]:
np.mean(votemper_ic_imuted[4,...])

In [ ]:
np.where(votemper_ic[votemper_ic == 'nan'])

In [ ]:
%reset

In [ ]:
np.mean(votemper_ic[0,...])